# ENSDF and RIPL Parsing

This notebook demonstrates the use of the ENSDF utilities to parse and generate the ENSDF csv files. It also demonstrates the use of the RIPL level parameteres file to cut-off the ENSDF csv file.

In [78]:
import sys
sys.path.append("../..")

import nucml.datasets as nuc_data
import nucml.ensdf.parsing_utilities as ensdf_parser

In [79]:
import importlib
importlib.reload(nuc_data)
importlib.reload(ensdf_parser)

<module 'nucml.ensdf.parsing_utilities' from '../..\\nucml\\ensdf\\parsing_utilities.py'>

# Searching for .dat Files

The ENSDF data is being extracted from the RIPL formated zXXX.dat files. 

In [26]:
# the `nuc_data` module contains all unique exfor elements
elements = nuc_data.exfor_elements

The `get_ripl_names()` function allows to get the path to all `.dat` files in the levels directory. We will use these to create our dataset. 

In [27]:
ripl_levels_dir = "../RIPL/levels/levels/"

In [28]:
names = ensdf_parser.get_ripl_dat_paths(ripl_levels_dir)

INFO:root:RIPL: Searching ../RIPL/levels/levels/ directory for .dat files...
INFO:root:RIPL: Finished. Found 118 .dat files.


# Extracting the Headers Only

We can now extract the header for each file which contains information we will be useful like the number of levels per isotope. 

In [29]:
saving_directory = "../CSV_Files/"

In [30]:
ensdf_parser.get_headers(names, saving_directory)

INFO:root:ENSDF: Extracting headers ...
INFO:root:ENSDF: Finished. Saved to ../CSV_Files/all_ensdf_headers_formatted.csv


# Generating Elemental Files

We now specify the directory we're the element-wise ensdf files will be stored. Currently, the .dat files are stored proton-wise. Whatever directory is specified, the `generate_elemental_ensdf()` function will create three directories where different types of files will be stored.

In [31]:
elemental_dir = "../Elemental_ENSDF/"

In [33]:
ensdf_parser.generate_elemental_ensdf(names, saving_directory, elemental_dir)

INFO:root:GEN UTILS: Directory does not exists. Creating...
INFO:root:GEN UTILS: Directory created.
INFO:root:ENSDF Elemental: Extracting ENSDF data per element with header...
INFO:root:GEN UTILS: Directory does not exists. Creating...
INFO:root:GEN UTILS: Directory created.
INFO:root:ENSDF Elemental: Removing header from ENSDF elemental files...
INFO:root:GEN UTILS: Directory does not exists. Creating...
INFO:root:GEN UTILS: Directory created.
INFO:root:ENSDF Elemental: Formatting files...
INFO:root:ENSDF Elemental: Finished formating data.


# Generating Ground State File

The elemental files contain all known nuclear levels, if for some reason you only need a ground state CSV file. This can be created using the `get_stable_states()` utility.

In [35]:
ensdf_parser.get_stable_states(names, saving_directory)

INFO:root:STABLE STATES: Extracting stable states from .dat files...
INFO:root:STABLE STATES: Formatting text file...
INFO:root:STABLE STATES: Finished.


# Generating ENSDF CSV File

With the elemental files created, we can know get a single ENSDF file containing all nuclear levels for all isotopes.

In [46]:
ensdf_parser.generate_ensdf_csv(saving_directory, elemental_dir)

INFO:root:ENSDF CSV: Creatign DataFrame with Basic ENSDF data ...
INFO:root:ENSDF CSV: Finished creating list of dataframes.


# Cutting ENSDF using RIPL CutOff Parameters

We can easily remove nuclear levels above the given parameteres using the the RIPL level-params.dat file. If you wish to create your own cut-off parameters be sure to modify the level-params.dat following the original formatting, otherwise this script will not work.

First we start by parsing the levels-params.data file from RIPL:

In [62]:
ripl_level_params_dir = "../RIPL/levels/"
ensdf_parser.get_level_parameters(ripl_level_params_dir, saving_directory=saving_directory)

INFO:root:ENSDF RIPL: Parsing and formatting level parameters...
INFO:root:ENSDF RIPL: Finished formating data. Converting to CSV...
INFO:root:ENSDF RIPL: Finished.


Now we can use the created cut-off csv file to remove nuclear levels from the previuosly created ensdf.csv file.

In [80]:
elemental_hf_dir = "../Elemental_ENSDF/Elemental_ENSDF_no_Header_F/"
ensdf_parser.generate_cutoff_ensdf(saving_directory, elemental_hf_dir)

INFO:root:ENSDF CutOff: Loading ENSDF and RIPL parameters...
INFO:root:ENSDF CutOff: Cutting off ENSDF...
INFO:root:ENSDF CutOff: Finished.
